<a href="https://colab.research.google.com/github/terri-pietka-ds/HCD-Imaging-Classification/blob/main/TP_Kaggle_HCD_Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Link to Kaggle

In [1]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
histopathologic_cancer_detection_path = kagglehub.competition_download('histopathologic-cancer-detection')

print('Data source import complete.')


100%|██████████| 6.31G/6.31G [00:54<00:00, 124MB/s]

Extracting files...


Data source import complete.


# Connect to drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Filter images and create file df

In [5]:
import os
import pandas as pd
import cv2
import numpy as np

# Paths to data
train_images = '/root/.cache/kagglehub/competitions/histopathologic-cancer-detection/train/'
label_csv = '/root/.cache/kagglehub/competitions/histopathologic-cancer-detection/train_labels.csv'

# Read labels
full_df = pd.read_csv(label_csv)
full_df['id'] = full_df['id'] + '.tif'
full_df['label'] = full_df['label'].astype(str)

# Filtering function to remove images that are mostly white or black
def filter_images(image_dir, image_ids, black_thresh=0.90, white_thresh=0.90):
    valid_ids = []

    for img_id in image_ids:
        img_path = os.path.join(image_dir, img_id)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            continue  # Skip unreadable images

        # Normalize pixel values to [0, 1]
        img = img / 255.0

        # Calculate black and white pixel ratios
        black_ratio = np.sum(img < 0.1) / img.size
        white_ratio = np.sum(img > 0.9) / img.size

        # Keep images that are NOT mostly black or white
        if black_ratio < black_thresh and white_ratio < white_thresh:
            valid_ids.append(img_id)

    return valid_ids

# Filter the images
valid_image_ids = filter_images(train_images, full_df['id'].tolist())
print(f"Number of valid images: {len(valid_image_ids)}")

# Update full_df to include only valid images
filtered_df = full_df[full_df['id'].isin(valid_image_ids)].reset_index(drop=True)
print(f"Updated dataframe shape: {filtered_df.shape}")



Number of valid images: 217330
Updated dataframe shape: (217330, 2)


# Store images to drive

In [9]:
import os
import shutil

# Source and destination paths
source_dir = '/root/.cache/kagglehub/competitions/histopathologic-cancer-detection/train/'
dest_dir = '/content/drive/MyDrive/Data/HCD/histopathologic_filtered_images/'

# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

# Copy valid images to Google Drive
for img_id in valid_image_ids:
    src_path = os.path.join(source_dir, img_id)
    dst_path = os.path.join(dest_dir, img_id)
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

print(f"Images copied to: {dest_dir}")


Images copied to: /content/drive/MyDrive/Data/HCD/histopathologic_filtered_images/


# Save df to drive

In [6]:
import pickle

# Save filtered DataFrame
filtered_df.to_csv('/content/drive/MyDrive/Data/HCD/data_frame/filtered_train_labels.csv', index=False)
print("CSV saved successfully.")

# Save valid image IDs
with open('/content/drive/MyDrive/Data/HCD/data_frame/valid_image_ids.pkl', 'wb') as f:
    pickle.dump(valid_image_ids, f)
print("Pickle file saved successfully.")

# Verify files in the directory
!ls /content/drive/MyDrive/Data/HCD



CSV saved successfully.
Pickle file saved successfully.
data_frame  filtered_train_labels.csv  histopathologic_filtered_images	valid_image_ids.pkl


In [7]:
# Correct Paths
label_csv = '/content/drive/MyDrive/Data/HCD/data_frame/filtered_train_labels.csv'
valid_ids_pickle = '/content/drive/MyDrive/Data/HCD/data_frame/valid_image_ids.pkl'
train_images = '/content/drive/MyDrive/Data/HCD/histopathologic_filtered_images/'

# Check file paths
print("Checking file paths...")
print(f"CSV exists: {os.path.exists(label_csv)}")
print(f"Pickle exists: {os.path.exists(valid_ids_pickle)}")
print(f"Image directory exists: {os.path.exists(train_images)}")



Checking file paths...
CSV exists: True
Pickle exists: True
Image directory exists: True
